In [1]:
import os
import cv2
import numpy as np
from scipy.signal import convolve2d

In [2]:
BASE_PATH = './archive/MMU-Iris-Database'
TEMPLATE_PATH = 'template'
os.makedirs(TEMPLATE_PATH, exist_ok=True)

In [3]:
REG_SUBJ, REG_EYE, REG_SHOT = 1, 'left', 'aeval1.bmp'
SAME_SHOT = 'bryanl1.bmp'
DIFF_SUBJ = 5

In [4]:
# 임계값
MAX_SHIFT = 32

In [5]:
def img_path(subj: int, eye: str, shot: str) -> str:
    return os.path.join(BASE_PATH, str(subj), eye, shot)

In [6]:
def load_iris(subj: int, eye: str, shot: str):
    path = img_path(subj, eye, shot)
    if not os.path.exists(path):
        raise FileNotFoundError(f'No such file: {path}')
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f'Failed to load image: {path}')
    return img

In [7]:
img = load_iris(REG_SUBJ, REG_EYE, REG_SHOT)
cv2.imshow('img', img)
cv2.waitKey(0)

-1

In [8]:
def detect_circles_hough(gray: np.ndarray):
    h, w = gray.shape
    blur = cv2.GaussianBlur(gray, ksize=(7, 7), sigmaX=1.5)

    # 동공 후보(작은 원)
    circles_pupil = cv2.HoughCircles(
        blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
        param1=120, param2=18, minRadius=15, maxRadius=int(0.18*min(h, w))
    )

    # 홍채 후보(큰 원)
    circles_iris = cv2.HoughCircles(
        blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
        param1=150, param2=28, minRadius=int(0.22*min(h, w)), maxRadius=int(0.45*min(h, w))
    )

    cx, cy = w // 2, h // 2
    r_in = int(0.14 * min(h, w))
    r_out = int(0.33 * min(h, w))

    if circles_pupil is not None:
        c = circles_pupil[0][0]
        cx, cy, r_in = int(c[0]), int(c[1]), max(int(c[2]), 10)

    if circles_iris is not None:
        c = circles_iris[0][0]
        # pupil/iris 중심 편차가 크면 pupil 중심을 유지
        cx = int(0.5*cx + 0.5*int(c[0]))
        cy = int(0.5*cy + 0.5*int(c[1]))
        r_out = max(int(c[2]), r_in + 15)

    r_out = max(r_out, r_in + 20)
    return (cx, cy), r_in, r_out

In [9]:
img = load_iris(REG_SUBJ, REG_EYE, REG_SHOT)
center, r_in, r_out = detect_circles_hough(img)
img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
cv2.circle(img, (center[0], center[1]), r_in, (255, 0, 0), 2)
cv2.circle(img, (center[0], center[1]), r_out, (0, 255, 0), 2)
cv2.imshow('img', img)
cv2.waitKey(0)

-1

In [10]:
import numpy as np
def normalize_iris(gray: np.ndarray, center, r_in, r_out, H=64, W=512) -> np.ndarray:
    h, w = gray.shape
    yy, xx = np.indices((H, W))

    theta = 2 * np.pi * xx / W
    r = r_in + (r_out - r_in) * (yy / (H - 1))

    X = (center[0] + r * np.cos(theta)).astype(np.int32)
    Y = (center[1] + r * np.sin(theta)).astype(np.int32)

    X = np.clip(X, a_min=0, a_max=w - 1)
    Y = np.clip(Y, a_min=0, a_max=h - 1)

    nor = gray[Y, X].astype(np.float32)

    return nor

In [11]:
img = load_iris(REG_SUBJ, REG_EYE, REG_SHOT)
center, r_in, r_out = detect_circles_hough(img)
nor = normalize_iris(img, center, r_in, r_out, H=64, W=512)
cv2.imshow('img', nor.astype(np.uint8))
cv2.waitKey(0)

-1

In [12]:
def log_gabor_id(W: int, f0: float, sigma_f: float) -> np.ndarray:
    freqs = np.fft.fftfreq(W)
    f = np.abs(freqs)
    G = np.zeros_like(f, dtype=np.float32)
    eps = 1e-9
    passband = f > 0
    G[passband] = np.exp(- (np.log(f[passband] / (f0 + eps)) ** 2) / (2 * (np.log(sigma_f) ** 2)))
    G[~passband] = 0.0

    return np.fft.fftshift(G).astype(np.float32)

In [13]:
def encode_iris(nor: np.ndarray, f0s=(0.05,0.10), sigma_f=1.5):
    H,W = nor.shape
    nor = (nor - np.mean()) / (nor.std() + 1e-6)
    codes = []
    for f0 in f0s:
        k = log_gabor_id(W, f0, sigma_f).reshape(1, -1)
        resp_r = convolve2d(nor, k, mode='same', boundary='wrap')
        resp_i = convolve2d(nor, np.roll(k, 1, axis=1), mode='same', boundary='wrap')
        codes.append((resp_r >= 0).astype(np.uint8))
        codes.append((resp_i >= 0).astype(np.uint8))
    code = np.stack(codes, axis=1)
    return code

In [14]:
img = load_iris(REG_SUBJ, REG_EYE, REG_SHOT)
center, r_in, r_out = detect_circles_hough(img)
nor = normalize_iris(img, center, r_in, r_out, H=64, W=512)
code = encode_iris(nor)

print(code)

TypeError: mean() missing 1 required positional argument: 'a'

In [ ]:
def simple_mask(nor: np.ndarray, low=5, high=250,eyelid_rows=5) -> np.ndarray:
    m = (nor > low) & (nor < high)
    m[:eyelid_rows, :] = 0
    m[-eyelid_rows:, :] = 0
    return m.astype(np.uint8)

In [ ]:
img = load_iris(REG_SUBJ, REG_EYE, REG_SHOT)
center, r_in, r_out = detect_circles_hough(img)
nor = normalize_iris(img, center, r_in, r_out, H=64, W=512)
code = encode_iris(nor)
mask = simple_mask(code)

In [ ]:
def save_template(subj: int, eye: str, code: np.ndarray, mask: np.ndarray, meta: dict):
    out = os.path.join(TEMPLATE_PATH, f'subj{subj}_{eye}.npz')
    np.savez_compressed(out, code=code, mask=mask, **meta)
    return out

In [ ]:
img = load_iris(REG_SUBJ, REG_EYE, REG_SHOT)
center, r_in, r_out = detect_circles_hough(img)
nor = normalize_iris(img, center, r_in, r_out, H=64, W=512)
code = encode_iris(nor)
mask = simple_mask(code)
meta = dict(center=np.array(center), r_in=r_in, r_out=r_out, H=nor.shape[0], W=nor.shape[1])
path = save_template(REG_SUBJ, REG_EYE, code, mask, meta)

In [ ]:
def register(subject: int, eye: str, shot: str):
    img = load_iris(subject, eye, shot)
    center, r_in, r_out = detect_circles_hough(img)
    nor = normalize_iris(img, center, r_in, r_out)
    code = encode_iris(nor)
    mask = simple_mask(code)
    meta = dict(center=np.array(center), r_in=r_in, r_out=r_out, H=nor.shape[0], W=nor.shape[1])
    path = save_template(subject, eye, code, mask, meta)
    return dict(template_path=path, center=center, r_in=r_in, r_out=r_out, shape=nor.shape)

In [ ]:
reg_info = register(REG_SUBJ, REG_EYE, REG_SHOT)
print(reg_info)